In [1]:
print("Analysing San Jose Animal Shelter Data")

Analysing San Jose Animal Shelter Data


In [2]:
import pandas as pd

df = pd.read_csv("../data/San_Jose_Animal_Shelter_FY23-24.csv")

# df.columns = prints names

# more verbose column info
df.nunique(axis=1)
df.count()
#df.info(verbose=True)

_id                 19239
AnimalID            19239
AnimalName          10263
AnimalType          19239
PrimaryColor        19239
SecondaryColor       7377
PrimaryBreed        19239
Sex                 19239
DOB                 12025
Age                 19239
IntakeDate          19239
IntakeCondition     19239
IntakeType          19239
IntakeSubtype       15500
IntakeReason          549
OutcomeDate         18229
OutcomeType         18232
OutcomeSubtype       8197
OutcomeCondition    17091
Crossing            14459
Jurisdiction        18625
LastUpdate          19239
dtype: int64

In [3]:
df.IntakeCondition.unique()
# df.OutcomeType.unique()
# RTO means return to owner
# RTF means return to field
# EUTH = euthanasia
# BEH R/M/U ? behavior issues?
# df.OutcomeCondition.unique()

array(['MED R', 'DEAD', 'MED M', 'HEALTHY', 'MED SEV', 'MED EMERG',
       'BEH U', 'AGGRESSIVE', 'FERAL', 'BEH M', 'BEH R', 'PREGNANT',
       'NURSING'], dtype=object)

In [4]:
# S/N clinic = spay/neuter clinic
df.IntakeType.unique()
# removed: disposal req, foster, disaster, s/n clinic

array(['STRAY', 'DISPO REQ', 'FOSTER', 'OWNER SUR', 'CONFISCATE',
       'EUTH REQ', 'RETURN', 'TRANSFER', 'DISASTER', 'S/N CLINIC',
       'WILDLIFE'], dtype=object)

In [15]:
intake_cols = ['AnimalID','IntakeType', 'IntakeCondition', 'IntakeDate']
intake = df[intake_cols]
print(intake.columns)
#deg = deg[['IntakeType', 'count']]
# removed: disposal req, foster, disaster, s/n clinic
intake = intake.loc[intake['IntakeType'].isin(['STRAY','OWNER SUR','CONFISCATE','EUTH REQ', 
                                                            'RETURN', 'TRANSFER', 'WILDLIFE'])]
intake.count()

Index(['AnimalID', 'IntakeType', 'IntakeCondition', 'IntakeDate'], dtype='object')


AnimalID           13095
IntakeType         13095
IntakeCondition    13095
IntakeDate         13095
dtype: int64

In [16]:
# this took so long....
# group the columns by intake type, and add a count column
condition_chart = intake.groupby('IntakeType').size().reset_index(name='counts')

#sort count descending
condition_chart = condition_chart.sort_values('counts', ascending=False)

condition_chart

,IntakeType,counts
4,STRAY,10959
6,WILDLIFE,741
2,OWNER SUR,575
0,CONFISCATE,426
3,RETURN,261
5,TRANSFER,86
1,EUTH REQ,47


In [17]:
# testing plotly
import plotly.express as px

fig = px.bar(condition_chart, x="IntakeType", y="counts", title="Intake Types", text_auto=True)
fig.show()
# restarting the entire notebook finally made the graph work btw

In [20]:
# pie chart for funsies
fig = px.pie(condition_chart, names="IntakeType", values="counts", title="Intake Types (%)")
fig.show()

In [11]:
# figure out correlation between intake condition and type
intake_comparison = intake_comparison.groupby(['IntakeType', 'IntakeCondition']).size().reset_index(name='counts')

intake_comparison = intake_comparison.sort_values('counts', ascending=False)
intake_comparison

AnimalID           13095
IntakeType         13095
IntakeCondition    13095
IntakeDate         13095
dtype: int64


,IntakeType,IntakeCondition,counts
42,STRAY,HEALTHY,4238
45,STRAY,MED R,2446
47,STRAY,NURSING,1586
46,STRAY,MED SEV,1085
44,STRAY,MED M,800
...,...,...,...
54,TRANSFER,MED EMERG,1
50,TRANSFER,BEH R,1
49,TRANSFER,BEH M,1
22,OWNER SUR,FERAL,1


In [18]:
fig2 = px.bar(intake_comparison, x="IntakeType", y="counts", color="IntakeCondition", title="Intake Breakdown")
fig2.show()

## Initial thoughts
It is obvious that the vast majority of animals in the shelter come from strays. I would assume that they are either found or reported in some public area. I am intersted in the fact that there is a substantial percentage of wildlife. Why is that? There is also a 4% of owners surrendering their pets, and a 3% being confiscated. I don't believe these to be exactly accurate, as it is possible that owners may also have released their pets and are taken in as strays. Either way, still interesting numbers. The 2% return is also worth looking at, though there doesn't seem to be much data to analyze. All in all still a low number. The euthanasia is not surprising.

It seems like there isn't much of a correlation between intake type and intake condition, other than obvious ones (eg euthansia/disposal means the animal is dead). It is unlikely that we will find major conclusions in these two graphs, but it is interesting to see the breakdown. 

We can see if we can find a correlation between intake types and outcome types, or something like that.